In [1]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_PATH = os.path.join(os.getcwd(), '../data/5_pts')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/5_pts')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/5_pts


In [3]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

horizontal_fov = 1.3962634

In [4]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [5]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Additional rotation  matrix to shift camera from +x to -z direction and make the x axis point to the right
    A = Rotation.from_euler('YZX', [-90, -90, 0], degrees=True).as_matrix()

    #Translation matrix in the world frame
    t = np.zeros((4,4))
    t[:3,:3] = np.matmul(r.T,A)
    t[:3,3] = -np.matmul(r.T,xyz)
    t[3,3] = 1

    print(t[:3,3])
    print(Rotation.from_matrix(t[:3,:3]).as_euler('xyz', degrees=True))
    
    return t

In [6]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    #Save image in processed folder
    img = cv2.imread(data_img)
    #Change img from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1       

[-1.25238254 -0.16252193  2.00009306]
[ 6.00208025e+01 -3.26319681e-02 -8.51775079e+01]
[-1.24809324  0.19264795  1.99994322]
[ 6.00273516e+01 -2.69912969e-02 -1.01356376e+02]
[-1.13299904  0.55982492  1.9997807 ]
[ 6.00343418e+01 -1.83947697e-02 -1.18898123e+02]
[-0.90480438  0.88190887  1.99965067]
[ 6.00396667e+01 -5.75338317e-03 -1.36872278e+02]
[-0.57399286  1.12575239  1.99963032]
[ 6.00401165e+01  4.62618101e-03 -1.55615011e+02]
[-0.22916258  1.24256642  1.99972779]
[ 6.00350070e+01  1.94522534e-02 -1.72202679e+02]
[0.20335226 1.24673589 1.99987472]
[6.00279935e+01 2.69122099e-02 1.68106195e+02]
[0.57053326 1.12638914 2.00009873]
[6.00174045e+01 3.59170035e-02 1.50464080e+02]
[0.86963871 0.91494682 2.00030965]
[6.00076479e+01 3.96220194e-02 1.33772142e+02]
[1.11705506 0.58805684 2.00058065]
[5.99952023e+01 4.24952686e-02 1.15100887e+02]
[1.24523618 0.20193228 2.00090403]
[5.99808624e+01 3.47886133e-02 9.65529965e+01]
[ 1.24466455 -0.20709315  2.00119086]
[5.99680842e+01 2.917412

In [7]:
transform_dictionary = {}
transform_dictionary["horizontal_fov"] = horizontal_fov
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile,indent=4)
